# TensorFlow Lite model optimization

TensorFlow Lite에서는 edge device에서 모델의 inference를 효율적으로 하기 위한 방법인 quantization과 pruning을 이용할 수 있는 toolkit을 제공하고 있습니다. 이번 실습에서는 해당 toolkit을 이용해 볼 것입니다.

더 자세한 내용은 document를 참고 해주세요.
- https://www.tensorflow.org/model_optimization/guide
- https://www.tensorflow.org/lite/performance/model_optimization
- TFLite https://www.tensorflow.org/lite/guide

## 0. What is TFLite?

TensorFlow Lite는 TensorFlow로 학습된 모델을 tflite이라는 포맷으로 바꿔 줌으로써 모바일 장치, 임베디드 장치 등의 가벼운 장치에서 모델을 inference 할 수 있게 해주는 플랫폼 이다.

<img src="https://user-images.githubusercontent.com/37704174/115159215-fccd0800-a0cc-11eb-817b-645f96c76966.png" width="400" height="400"/>  


TensorFlow는 모델을 파일로 저장할 때는 [Protocol Buffer](https://developers.google.com/protocol-buffers)라는 포맷으로 저장을 하고, TensorFlow Lite의 모델은 [FlatBuffer](https://google.github.io/flatbuffers/)라는 포맷으로 저장을 합니다. 두 포맷의 가장 큰 차이점은 FlatBuffer는 serialized 파일을 추가적인 메모리 할당 없이 deserialize를 할 수 있기 때문에 빠르게 파일에 있는 데이터에 접근할 수 있다.


TFLite 모델로 변환을 할 때, quantization과 pruning 등의 optimization을 할 수 있는데, 이 때의 장점은 다음과 같다.

- Size reduction
  - smaller storage size, smaler download size, less memory usage
- Latency reduction
  - computation을 줄여 줌으로써 inference에 걸리는 시간 감소
- Accelerator compatibility
  - Edge TPU와 같이 특정 hardware는 각각의 quantization 요구 사항을 만족해야 함

Colab Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
"""
Change directory to where this file is located
"""
%cd '/content/drive/MyDrive/Colab Notebooks/Ambient AI/Lesson/Lesson_6'

/content/drive/MyDrive/Colab Notebooks/Ambient AI/Lesson/Lesson_6


tensorflow 최신 버전 설치

In [3]:
!pip install -q --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 21.3 MB/s eta 0:00:00


In [14]:
import os
import numpy as np
import zipfile
import tempfile
import tensorflow as tf
#assert float(tf.__version__[:3]) >= 2.6

tf.random.set_seed(1111)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [15]:
tf.__version__

'2.13.0'

GPU Setup

In [16]:
"""
Make sure your runtime type is GPU!
"""
physical_devices = tf.config.list_physical_devices('GPU')
print('Num_GPUs:{}, List:{}'.format(len(physical_devices), physical_devices))

Num_GPUs:1, List:[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [17]:
gpu_growth = False

if gpu_growth:
    physical_devices = tf.config.list_physical_devices('GPU')
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except:
        # Invalid device or cannot modify virtual devices once initialized.
        pass

## 1. Quantization
### 1-1. [Post-training quantization](https://blog.tensorflow.org/2019/06/tensorflow-integer-quantization.html)

post-training quantization은 학습을 한 다음에 quantization을 하는 방법이다. quantization aware training보다는 accuracy가 좀 떨어질 수 있지만, 학습된 모델을 가져와서 쉽게 쓸 수 있다. tensorflow에서 다음과 같은 방법들을 제공하고 있다.


<img src="https://user-images.githubusercontent.com/37704174/114367164-2232b100-9bb7-11eb-81e7-b9b8dec09de5.png" width="700" height="700"/>  
- dynamic range quantization: weight은 8 bit integer로 저장. inference 시에는 8bit float로 바뀜. activation은 range에 따라서(dynamic) 8 bit float으로 저장


나중에 Edge TPU가 있는 Coral board를 사용할 것이기 때문에, post-training integer quantization을 다뤄 볼 것이다.


먼저 quantizaion을 할 간단한 모델을 학습한다.

In [18]:
# Load MNIST dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Normalize the input image
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28)),
    tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile and Train
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=False),
              metrics=['accuracy'])
model.fit(
  train_images,
  train_labels,
  epochs=10,
  validation_split=0.1
)

metrics = model.evaluate(test_images, test_labels)

4422102/4422102 [==============================] - 0s 0us/step
Epoch 1/10
1688/1688 [==============================] - 18s 4ms/step - loss: 0.4790 - accuracy: 0.8295 - val_loss: 0.3816 - val_accuracy: 0.8648
Epoch 2/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3351 - accuracy: 0.8811 - val_loss: 0.3323 - val_accuracy: 0.8780
Epoch 3/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2970 - accuracy: 0.8943 - val_loss: 0.2914 - val_accuracy: 0.8947
Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2725 - accuracy: 0.9034 - val_loss: 0.2808 - val_accuracy: 0.8978
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2531 - accuracy: 0.9094 - val_loss: 0.2732 - val_accuracy: 0.8982
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2350 - accuracy: 0.9161 - val_loss: 0.2967 - val_accuracy: 0.8942
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - lo

In [19]:
print(f"Model test loss: {metrics[0]:.03f} | test accuracy: {metrics[1]*100:.02f}%")

Model test loss: 0.275 | test accuracy: 90.42%


학습된 모델을 변환하기 위해서는 [TensorFlow Lite converter](https://www.tensorflow.org/lite/convert)라는 것을 이용해야 합니다. 이것은 TensorFlow 모델을 TensorFlow Lite 모델로 바꿔 줍니다.

다음과 같이 tflite 모델로 변환할 수 있습니다.

In [20]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

변환할 때 아무런 옵션을 주지 않았으므로 이 모델은 단순히 32-bit float tflite 모델입니다.

### Post-training integer quantization


<img src="https://user-images.githubusercontent.com/37704174/114906428-f9c7e280-9e54-11eb-9038-9854d5c8d741.png" width="700" height="700"/>  
<img src="https://user-images.githubusercontent.com/37704174/114907026-9e4a2480-9e55-11eb-980f-55999688c260.png" width="500" height="500"/>  


다음 셀을 실행하면 quantization이 완료 된다.

In [21]:
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8 # or tf.int8
converter.inference_output_type = tf.uint8 # or tf.int8

post_quant_tflite_model = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


evaluate을 위한 helper function이다. input type을 integer로 했으므로, quantize를 한 다음에 inference를 한다.

In [22]:
from tqdm import tqdm

def evaluate_model(interpreter):
    global test_images

    input_details = interpreter.get_input_details()[0]
    input_index = input_details["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    # Run predictions on every image in the "test" dataset.
    prediction_digits = []
    for test_image in tqdm(test_images):
        # quantize input
        if input_details['dtype'] == np.uint8:
            input_scale, input_zero_point = input_details["quantization"]
            test_image = test_image / input_scale + input_zero_point

        # Pre-processing: add batch dimension
        test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
        interpreter.set_tensor(input_index, test_image)

        # Run inference.
        interpreter.invoke()

        # Post-processing: remove batch dimension and find the digit with highest
        # probability.
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)

    print()
    # Compare prediction results with ground truth labels to calculate accuracy.
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == test_labels).mean()
    return accuracy, output

tflite 모델로 inference를 하려면 [`Interpreter`](https://www.tensorflow.org/api_docs/python/tf/lite/Interpreter)를 이용한다. `model_content` 대신에 `model_path`를 이용하면 tflite 파일을 불러올 수도 있다.  
inference를 하기 위해서는  
interpreter 생성 &#8594; `allocate_tensor` &#8594; `set_tensor` &#8594; `invoke` 순으로 실행한다.  
지금 코드는 inference 할 때 image를 하나씩 처리하는데, batch 단위로 처리하려면 `resize_tensor_input`을 이용한다.

In [23]:
# 32bit float tflite 모델로 interpreter 만듦
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# inference 할 때 input batch 수를 설정하하는 법
# input_details = interpreter.get_input_details()[0]
# interpreter.resize_tensor_input(input_details["index"], [10, 28, 28]) # <-- 추가

interpreter.allocate_tensors()
tflite_test_accuracy, _ = evaluate_model(interpreter)

# 8bit integer tflite
post_quant_interpreter = tf.lite.Interpreter(model_content=post_quant_tflite_model)
post_quant_interpreter.allocate_tensors()
post_quant_tflite_test_accuracy, output = evaluate_model(post_quant_interpreter)

100%|██████████| 10000/10000 [00:03<00:00, 2977.02it/s]


100%|██████████| 10000/10000 [00:01<00:00, 5947.95it/s]

`get_input_details`, `get_output_details`를 통해 input과 output의 type 및 quantization 정보를 알 수 있다.

In [24]:
post_quant_interpreter.get_input_details()

[{'name': 'serving_default_input_1:0',
  'index': 0,
  'shape': array([ 1, 28, 28], dtype=int32),
  'shape_signature': array([-1, 28, 28], dtype=int32),
  'dtype': numpy.uint8,
  'quantization': (0.003921568859368563, 0),
  'quantization_parameters': {'scales': array([0.00392157], dtype=float32),
   'zero_points': array([0], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [25]:
post_quant_interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 24,
  'shape': array([ 1, 10], dtype=int32),
  'shape_signature': array([-1, 10], dtype=int32),
  'dtype': numpy.uint8,
  'quantization': (0.00390625, 0),
  'quantization_parameters': {'scales': array([0.00390625], dtype=float32),
   'zero_points': array([0], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

실제로 inference output이 정수임을 확인할 수 있다.

In [26]:
print(output())

[[  2   0   4   0   0 117   0 117  15   2]]


float 32 모델과 accuracy를 비교해 보자.

In [27]:
print('TFLite test accuracy:', tflite_test_accuracy)
print('Post-training Quant TFLite test_accuracy:', post_quant_tflite_test_accuracy)

TFLite test accuracy: 0.9042
Post-training Quant TFLite test_accuracy: 0.9052


모델을 파일로 저장해서 실제로 모델 크기가 줄어들었는지 확인해 보자.

In [28]:
# create temp file
_, tflite_file = tempfile.mkstemp('.tflite')
_, post_quant_tflite_file = tempfile.mkstemp('.tflite')


with open(tflite_file, 'wb') as f:
    f.write(tflite_model)

with open(post_quant_tflite_file, 'wb') as f:
    f.write(post_quant_tflite_model)


tflite_size = os.path.getsize(tflite_file) / float(2**20)
quantized_size = os.path.getsize(post_quant_tflite_file) / float(2**20)

print(f"Float model: {tflite_size:.03f} MB")
print(f"Quantized model: {quantized_size:.03f} MB")
print(f"Quantized model: {quantized_size/tflite_size:.02f} x Float model size", )

Float model: 0.082 MB
Quantized model: 0.025 MB
Quantized model: 0.31 x Float model size


### 실습: TensorFlow에서 제공하는 pretrain model 가져와서 post-training quantization 해보기

TensorFlow에서는 여러 pretrained model을 제공한다. 아무거나 하나 가져와서 post-training quantization을 해보자.  

In [29]:
# pretrain 모델 import 하기.
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from PIL import Image
from IPython.display import display

다음의 test image를 이용할 것이다. quantization을 할 때, representative data로는 이 이미지 하나만 이용해도 된다.

In [30]:
url = "https://cdn.education.com/files/526001_527000/526114/file_526114.jpg"
os.system("curl " + url + " > balloon.jpg")
img_path = 'balloon.jpg'
img = image.load_img(img_path, target_size=(224, 224))
display(img)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x) # 각 모델을 학습할 때 preprocess했던 방법을 사용

evaluate 할 때는 다음 함수를 이용

In [31]:
def evaluate_model_practice(interpreter, test_images):

    input_details = interpreter.get_input_details()[0]
    input_index = input_details["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    # Run predictions on every image in the "test" dataset.
    prediction_digits = []
    for test_image in tqdm(test_images):
        # quantize input
        if input_details['dtype'] == np.uint8:
            input_scale, input_zero_point = input_details["quantization"]
            test_image = test_image / input_scale + input_zero_point

        # Pre-processing: add batch dimension
        test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
        interpreter.set_tensor(input_index, test_image)

        # Run inference.
        interpreter.invoke()

        preds = interpreter.tensor(output_index)
        preds = np.expand_dims(preds()[0], axis=0)

    return preds

In [32]:
# 1. import한 pretrain 모델 가져오기
imagenet = ResNet50(weights='imagenet')

preds = imagenet.predict(x)
# decode the results into a list of tuples (class, description, probability)
print('Predicted:', decode_predictions(preds, top=3)[0])

35363/35363 [==============================] - 0s 0us/step
Predicted: [('n02782093', 'balloon', 0.9537052), ('n04023962', 'punching_bag', 0.03246281), ('n03544143', 'hourglass', 0.0014412437)]


크기 비교를 위한 32bit float tflite 모델

In [33]:
converter = tf.lite.TFLiteConverter.from_keras_model(imagenet)
imagenet_tflite = converter.convert()

In [34]:
# 2. full integer post-training quantization
import numpy as np
def representative_data_gen():
    yield [x] # test할 이미지 1개만 사용

converter = tf.lite.TFLiteConverter.from_keras_model(imagenet)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

post_quant_imagenet_tflite = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [35]:
# 3. make interpreter and do inference
# tflite 모델로 interpreter 만듦
post_quant_imagenet_interpreter = tf.lite.Interpreter(model_content=post_quant_imagenet_tflite)

post_quant_imagenet_interpreter.allocate_tensors()

preds = evaluate_model_practice(post_quant_imagenet_interpreter, x)
print('Predicted:', decode_predictions(preds, top=3)[0])

100%|██████████| 1/1 [00:00<00:00,  6.56it/s]

Predicted: [('n02782093', 'balloon', 244), ('n04023962', 'punching_bag', 8), ('n03942813', 'ping-pong_ball', 1)]


In [36]:
# Measure sizes of models.
_, imagenet_tflite_file = tempfile.mkstemp('.tflite')
_, post_quant_imagenet_tflite_file = tempfile.mkstemp('.tflite')


with open(tflite_file, 'wb') as f:
    f.write(imagenet_tflite)

with open(post_quant_tflite_file, 'wb') as f:
    f.write(post_quant_imagenet_tflite)


tflite_qat_size = os.path.getsize(tflite_file) / float(2**20)
quantized_size = os.path.getsize(post_quant_tflite_file) / float(2**20)

print(f"Float model: {tflite_qat_size:.03f} MB")
print(f"Quantized model: {quantized_size:.03f} MB")
print(f"Quantized model: {quantized_size/tflite_qat_size:.02f} x Float model size", )

Float model: 97.425 MB
Quantized model: 25.074 MB
Quantized model: 0.26 x Float model size


### 1-2. [Quantization Aware Training(QAT)](https://blog.tensorflow.org/2020/04/quantization-aware-training-with-tensorflow-model-optimization-toolkit.html)

수업 시간에 배웠 듯이 QAT는 나중에 quantization을 할 것이라는 것을 가정하고 학습하는 것이다. scratch부터 학습해도 되지만, 이미 학습된 모델을 fine tuning 하는 것이 좋다고 한다.

In [37]:
!pip install -q tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 46.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.1 which is incompatible.
tensorflow 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.25.1 which is incompatible.


In [38]:
import tensorflow_model_optimization as tfmot

### Quantize whole model

Edge TPU와 같이 fully quantized 모델이 필요한 경우에는 전체 모델에 대해 QAT를 한다.

위에서 학습한 float32모델로 QAT를 해볼 것이다.`quantize_model`함수를 이용한다.

In [39]:
quant_aware_model = tfmot.quantization.keras.quantize_model(model)

# `quantize_model` requires a recompile.
quant_aware_model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])

quant_aware_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 28, 28)            3         
 yer)                                                            
                                                                 
 quant_reshape (QuantizeWra  (None, 28, 28, 1)         1         
 pperV2)                                                         
                                                                 
 quant_conv2d (QuantizeWrap  (None, 28, 28, 16)        195       
 perV2)                                                          
                                                                 
 quant_max_pooling2d (Quant  (None, 14, 14, 16)        1         
 izeWrapperV2)                                                   
                                                                 
 quant_conv2d_1 (QuantizeWr  (None, 14, 14, 32)        4

layer에 quant가 붙은 것을 확인할 수 있다. 그 다음으로는, data의 일부를 이용해서 fine tuning 한다.

In [40]:
# data 일부를 이용해서 fine tuning
train_images_subset = train_images[0:1000] # out of 60000
train_labels_subset = train_labels[0:1000]

quant_aware_model.fit(train_images, train_labels, batch_size=500, epochs=10, validation_split=0.1)
model.fit(train_images, train_labels, batch_size=500, epochs=10, validation_split=0.1)

Epoch 1/10
108/108 [==============================] - 4s 14ms/step - loss: 0.1604 - accuracy: 0.9435 - val_loss: 0.2369 - val_accuracy: 0.9152
Epoch 2/10
108/108 [==============================] - 1s 11ms/step - loss: 0.1525 - accuracy: 0.9459 - val_loss: 0.2394 - val_accuracy: 0.9152
Epoch 3/10
108/108 [==============================] - 1s 11ms/step - loss: 0.1484 - accuracy: 0.9479 - val_loss: 0.2362 - val_accuracy: 0.9158
Epoch 4/10
108/108 [==============================] - 1s 11ms/step - loss: 0.1490 - accuracy: 0.9479 - val_loss: 0.2384 - val_accuracy: 0.9175
Epoch 5/10
108/108 [==============================] - 1s 11ms/step - loss: 0.1445 - accuracy: 0.9498 - val_loss: 0.2371 - val_accuracy: 0.9123
Epoch 6/10
108/108 [==============================] - 1s 11ms/step - loss: 0.1413 - accuracy: 0.9503 - val_loss: 0.2450 - val_accuracy: 0.9170
Epoch 7/10
108/108 [==============================] - 1s 11ms/step - loss: 0.1389 - accuracy: 0.9514 - val_loss: 0.2375 - val_accuracy: 0.9145

아직 quantization을 하지 않았으므로 weight은 실수이다.

In [41]:
quant_aware_model.layers[2].get_weights()

[array([[[[-6.8826921e-02, -5.4891080e-01, -4.2403153e-01,
           -4.1094041e-01, -2.4634248e-02, -3.2218277e-01,
           -3.1302425e-01,  1.4665209e-01,  2.4655087e-01,
           -3.1958240e-01,  3.0709726e-01,  3.7575367e-01,
            3.3023095e-01,  2.0531370e-01,  1.9645235e-01,
           -3.1541321e-01]],
 
         [[-4.9192399e-01,  2.3133552e-01,  5.3209895e-01,
           -1.0223347e+00,  8.8427849e-02, -3.4598672e-01,
            1.8345673e-02,  1.7164515e-01, -1.9334093e-01,
            3.1818181e-01,  4.9494794e-01, -6.1163012e-02,
           -5.6085205e-01, -2.7108526e-01,  1.2065147e-02,
            3.0488223e-01]],
 
         [[ 1.7502323e-01,  2.6479787e-01,  1.0021162e-01,
           -3.3824253e-01,  2.9271046e-02, -5.4884422e-01,
            2.9521054e-01,  1.3859983e-01, -2.5927106e-01,
            2.0895094e-02,  2.6244125e-01, -4.4170880e-01,
            2.6952827e-01, -6.2294757e-01, -3.9626807e-01,
            4.2763150e-01]]],
 
 
        [[[-7.21824

실수 weight이므로 QAT를 한다고 해서 accuracy가 떨어지지 않는다.

In [42]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, quant_aware_model_accuracy = quant_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('QAT test accuracy:', quant_aware_model_accuracy)

Baseline test accuracy: 0.913100004196167
QAT test accuracy: 0.9121999740600586


이제 QAT로 학습한 모델을 quantization 해보자. post-training quantization과 비슷하게 TFLiteConverter를 이용하면 된다.

In [43]:
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8 # or tf.int8
converter.inference_output_type = tf.uint8 # or tf.int8


QAT_tflite_model = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


quantization을 해도 accuracy가 떨어지지 않는 것을 알 수 있다.

In [44]:
QAT_interpreter = tf.lite.Interpreter(model_content=QAT_tflite_model)
QAT_interpreter.allocate_tensors()

QAT_test_accuracy, output = evaluate_model(QAT_interpreter)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Post-training Quantized test accuracy:', post_quant_tflite_test_accuracy)
print('Quantization-Aware Training test accuracy:', QAT_test_accuracy)


100%|██████████| 10000/10000 [00:01<00:00, 5663.64it/s]


Baseline test accuracy: 0.913100004196167
Post-training Quantized test accuracy: 0.9052
Quantization-Aware Training test accuracy: 0.9127


마찬가지로 크기가 1/4로 줄어든 것을 알 수 있다.

In [45]:
# create temp file
_, QAT_tflite_file = tempfile.mkstemp('.tflite')

with open(QAT_tflite_file, 'wb') as f:
    f.write(QAT_tflite_model)


quantized_size = os.path.getsize(QAT_tflite_file) / float(2**20)

print(f"Float tflite model: {tflite_size:.03f} MB")
print(f"Quantized QAT model: {quantized_size:.03f} MB")
print(f"Quantized QAT model: {quantized_size/tflite_size:.02f} x Float model size", )

Float tflite model: 0.082 MB
Quantized QAT model: 0.026 MB
Quantized QAT model: 0.32 x Float model size


### Quantize some layers

Edge TPU 등 특별한 하드웨어가 아닌 경우에는 선택적으로 layer를 quantization할 수도 있다.

**Tips for better accuracy**
* from scratch보다는 fine tuning
* 뒤쪽 layer를 quantization 하기
* 특별히 중요한 layer는 quantization 피하기


`quantize_annotate_layer` 함수를 이용해서 특정 layer를 quantization할 것이라고 표시한다.

In [46]:
model.layers

In [47]:
model.layers[-1].name

'dense'

In [48]:
# Helper Function
def apply_quantization_to_dense(layer):
    # if isinstance(layer, tf.keras.layers.Dense)
    if layer.name == 'dense':
        return tfmot.quantization.keras.quantize_annotate_layer(layer)
    else:
        return layer

`clone_model` 함수와 helper function을 이용해서 annotated model을 만들고, `quantize_apply`함수를 이용해서 QAT를 위한 모델을 만들면 된다.

In [49]:
annotated_model = tf.keras.models.clone_model(
    model,
    clone_function=apply_quantization_to_dense,
)
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
quant_aware_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 32)          0         
 g2D)                                                            
                                                                 
 quant_flatten (QuantizeWra  (None, 1568)              1

만약 fine tuning이 아니라 처음부터 모델을 만드는 것이면 다음과 같이 모델을 만들면 된다.

**Sequential example**

In [50]:
annotated_model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(28, 28)),
  tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tfmot.quantization.keras.quantize_annotate_layer(tf.keras.layers.Dense(10))
])
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

quant_aware_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 12)        0         
 g2D)                                                            
                                                                 
 quant_flatten_1 (QuantizeW  (None, 2028)              1         
 rapperV2)                                                       
                                                                 
 quant_dense_1 (QuantizeWra  (None, 10)                20295     
 pperV2)                                                         
                                                      

**Functional example**

In [51]:
inputs = tf.keras.Input(shape=(28, 28))
x = tf.keras.layers.Reshape(target_shape=(28, 28, 1))(inputs)
x = tfmot.quantization.keras.quantize_annotate_layer(tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'))(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(10)(x)

annotated_model = tf.keras.Model(inputs=inputs, outputs=outputs)

quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

quant_aware_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 quant_reshape_2 (QuantizeW  (None, 28, 28, 1)         1         
 rapperV2)                                                       
                                                                 
 quant_conv2d_3 (QuantizeWr  (None, 26, 26, 12)        147       
 apperV2)                                                        
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 13, 13, 12)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 2028)              0         
                                                             

## 2. [Pruning](https://blog.tensorflow.org/2019/05/tf-model-optimization-toolkit-pruning-API.html)

pruning은 불필요한(0에 가까운) weight을 0으로 만들어 없애면서 optimize를 한다.

`tfmot.sparsity.keras.prune_low_magnitude`을 이용해서 모델을 만든다. 이 때, pruning 방식인 `tfmot.sparsity.keras.PolynomialDecay` 을 parameter로 넘겨준다. 다음과 같은 hyperparemeter가 있다.
* `initial_sparsity`: pruning을 시작할 때의 sparsity를 몇으로 할 지
* `final_sparsity`: pruning을 끝낼 때의 sparsity를 몇으로 할 지
* `begin_step`: pruning을 언제부터 진행할 지 (batch 단위의 step)
* `end_step`: pruning을 언제 끝낼 지

pruning을 하기 전에 baseline model을 저장하겠다. (크기 비교시 사용)

In [52]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.913100004196167
Saved baseline model to: /tmp/tmpwe2dv4nl.h5


<ipython-input-52-98cb43f3aa6b>:7: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, keras_file, include_optimizer=False)


In [53]:
# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set.

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshap  (None, 28, 28, 1)         1         
 e (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_conv2d  (None, 28, 28, 16)        306       
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_max_po  (None, 14, 14, 16)        1         
 oling2d (PruneLowMagnitude                                      
 )                                                               
                                                                 
 prune_low_magnitude_conv2d  (None, 14, 14, 32)        9250      
 _1 (PruneLowMagnitude)                                          
                                                        

`fit`을 할 때, `tfmot.sparsity.keras.UpdatePruningStep`을 callback으로 불러야 한다

In [54]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


  6/422 [..............................] - ETA: 4s - loss: 0.1428 - accuracy: 0.9531   

422/422 [==============================] - 11s 12ms/step - loss: 0.2664 - accuracy: 0.9039 - val_loss: 0.3634 - val_accuracy: 0.8652
Epoch 2/2
422/422 [==============================] - 3s 8ms/step - loss: 0.3008 - accuracy: 0.8925 - val_loss: 0.3028 - val_accuracy: 0.8918


baseline 모델과 accuracy를 비교해 보자.

In [55]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.913100004196167
Pruned test accuracy: 0.8867999911308289


pruned model의 크기가 줄어들었는 지 확인해 보자.

위에 summary를 보면 pruning을 하기 위해 non-trainable parameter가 생긴 것을 알 수 있다. `tfmot.sparsity.keras.strip_pruning`을 이용해서 pruning에 사용한 variable을 제거해 준다.

In [56]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

<ipython-input-56-11ad47b980ba>:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: /tmp/tmpoo8dflt8.h5


tflite 으로 저장

In [57]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
    f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

Saved pruned TFLite model to: /tmp/tmp9_e4d8y8.tflite


pruning을 해도 weight matrix의 크기는 그대로이다. 하지만 matrix의 값이 대부분 0이기 때문에 감소된 크기를 확인하려면 실제로 zip파일 등으로 압축하는 과정을 거쳐야 한다.

In [58]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)
    return os.path.getsize(zipped_file)

In [59]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 78868.00 bytes
Size of gzipped pruned Keras model: 26661.00 bytes
Size of gzipped pruned TFlite model: 25448.00 bytes


### Combine pruning and quantization
pruning을 한 다음에 post-training quantization까지 하면 크기를 더 줄일 수 있다.

In [60]:
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
    f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))
print(f"Size of optimized model is {get_gzipped_model_size(quantized_and_pruned_tflite_file)/get_gzipped_model_size(keras_file):.02f} of the baseline model")

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Saved quantized and pruned TFLite model to: /tmp/tmpzntp2q85.tflite
Size of gzipped baseline Keras model: 78868.00 bytes
Size of gzipped pruned and quantized TFlite model: 9604.00 bytes
Size of optimized model is 0.12 of the baseline model


baseline 과 accuracy를 비교해 보자

In [61]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy, _ = evaluate_model(interpreter)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Pruned and quantized TFLite test_accuracy:', test_accuracy)

100%|██████████| 10000/10000 [00:01<00:00, 5816.77it/s]


Baseline test accuracy: 0.913100004196167
Pruned and quantized TFLite test_accuracy: 0.8866


QAT와 비슷하게, pruning도 layer를 선택적으로 pruning 할 수 있다. 이 부분은 [document](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide)를 참고